## Configure and installing libraries

In [49]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [ ]:
import os
import json

# Create .kaggle directory if it doesn't exist
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

# Move kaggle.json to the created directory
for fn in uploaded.keys():
    os.rename(fn, "/root/.kaggle/" + fn)

# Set permissions for the kaggle.json file
os.chmod("/root/.kaggle/kaggle.json", 600)

In [ ]:
# !pip install git+https://github.com/huggingface/datasets.git
# !pip install git+https://github.com/huggingface/transformers.git
!pip install jiwer
!pip install torchaudio
!pip install librosa
!pip install kagglehub tqdm
# Monitor the training process
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.0 MB/s eta 0:00:00


In [51]:
!pip install evaluate opensmile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 33.3 MB/s eta 0:00:00


## Getting Dataset

In [ ]:
import os
import glob
import pandas as pd
import kagglehub

# --------------------------
# STEP 1: Download datasets
# --------------------------

#English
RAVDESS_path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")
tess_path = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")
cremad_path = kagglehub.dataset_download("ejlok1/cremad")
#Arabic
eyase_path = kagglehub.dataset_download("mohammedhassanain/eyase-dataset")


print("Datasets downloaded!")
print("RAVDESS_path", RAVDESS_path)
print("cremad_path", cremad_path)
print("TESS_path:", tess_path)
print("EYASE path:", eyase_path)


Datasets downloaded!
RAVDESS_path /kaggle/input/ravdess-emotional-speech-audio
cremad_path /kaggle/input/cremad
TESS_path: /kaggle/input/toronto-emotional-speech-set-tess
EYASE path: /kaggle/input/eyase-dataset


In [ ]:
!cp /content/drive/MyDrive/usamah/emotion/KEDAS -r /content/ #1

^C


In [50]:
#Arabic
# !cp /content/drive/MyDrive/usamah/emotion/KEDAS -r /content/ #1
!cp /kaggle/input/eyase-dataset/EYASE/ -r /content/ #2
!mv /content/EYASE /content/eyase
!cp /content/drive/MyDrive/usamah/emotion/remake -r /content/ #3
!mv /content/remake /content/baved


#English
!cp /kaggle/input/ravdess-emotional-speech-audio -r /content/ #1
!mv /content/ravdess-emotional-speech-audio /content/ravd
!cp /kaggle/input/cremad -r /content/ #2
!cp /kaggle/input/toronto-emotional-speech-set-tess -r /content/ #3
!mv /content/toronto-emotional-speech-set-tess /content/tess

### Setting Arabic

In [ ]:
import os

import pandas as pd
# Path to your remake dataset
REMAKE_DATA_PATH = "/content/baved"

# Emotion mapping
REMAKE_EMOTIONS = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}

file_names = []
file_emotion = []
file_gender = []
data = []

# Loop through dataset
for folder in os.listdir(REMAKE_DATA_PATH):
    folder_path = os.path.join(REMAKE_DATA_PATH, folder)
    if not os.path.isdir(folder_path):
        continue

    for file in os.listdir(folder_path):
        if not file.endswith(".wav"):
            continue

        file_path = os.path.join(folder_path, file)

        # Extract emotion (before "__")
        emotion = file.split("__")[0]

        # Encode emotion
        if emotion in REMAKE_EMOTIONS.values():
            emotion_encoded = list(REMAKE_EMOTIONS.keys())[list(REMAKE_EMOTIONS.values()).index(emotion)]
        else:
            continue  # Skip unknown emotions

        # Extract gender (f or m after "__")
        try:
            parts = file.split("__")[1].split("-")
            gender = "female" if parts[1] == "f" else "male"
        except:
            gender = "unknown"

        file_emotion.append(emotion_encoded)
        file_names.append(file)
        file_gender.append(gender)

        data.append({
            "Name": file_path,
            "Emotion": emotion_encoded,
        })

print(f"Loaded {len(data)} files")

REMAKE_Data = pd.DataFrame(data)
REMAKE_Data.to_csv("REMAKE.csv",  index=False)
REMAKE_Data.head()



Loaded 1935 files


,Name,Emotion
0,/content/baved/4/joy_excited__4-m-20-4-2-1226.wav,1
1,/content/baved/4/calm_content__56-f-40-4-1-115...,4
2,/content/baved/4/sadness__55-m-16-4-0-1320.wav,2
3,/content/baved/4/sadness__55-m-16-4-0-1317.wav,2
4,/content/baved/4/joy_excited__47-m-20-4-2-1256...,1


In [ ]:
import os

eyase_DATA_PATH = "/content/eyase"

# New emotion mapping
EYASE_EMOTIONS = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}

# Old dataset emotion tags → new labels
OLD2NEW = {
    "ang": "anger_fear",
    "hap": "joy_excited",
    "sad": "sadness",
    "neu": "calm_content"   # map 'neutral' to 'calm_content'
}

file_names = []
file_emotion = []
file_gender = []
data = []

for gender_folder in os.listdir(eyase_DATA_PATH):
    folder_path = os.path.join(eyase_DATA_PATH, gender_folder)
    if not os.path.isdir(folder_path):
        continue

    gender = "female" if gender_folder.lower().startswith("female") else "male"

    for file in os.listdir(folder_path):
        if not file.endswith(".wav"):
            continue

        file_path = os.path.join(folder_path, file)

        # Extract old emotion (e.g., fm01_ang (10).wav → "ang")
        try:
            emotion_old = file.split("_")[1].split()[0]  # ang, hap, neu, sad
        except:
            continue

        if emotion_old not in OLD2NEW:
            continue

        emotion_new = OLD2NEW[emotion_old]

        # Encode new emotion
        emotion_encoded = list(EYASE_EMOTIONS.keys())[list(EYASE_EMOTIONS.values()).index(emotion_new)]

        file_emotion.append(emotion_encoded)
        file_names.append(file)
        file_gender.append(gender)

        data.append({
            "Name": file_path,
            "Emotion": emotion_encoded,
        })

print(f"Loaded {len(data)} files from EYASE")
eyase_Data = pd.DataFrame(data)
eyase_Data.to_csv("eyase.csv",  index=False)
eyase_Data.head()




Loaded 579 files from EYASE


,Name,Emotion
0,/content/eyase/Female03/fm03_sad (3).wav,2
1,/content/eyase/Female03/fm03_neu (3).wav,4
2,/content/eyase/Female03/fm03_neu (10).wav,4
3,/content/eyase/Female03/fm03_neu (4).wav,4
4,/content/eyase/Female03/fm03_hap (4).wav,1


In [ ]:
import os

KEDAS_DATA_PATH = '/content/drive/MyDrive/usamah/emotion/KEDAS/Data'

# Unified emotion mapping
KEDAS_EMOTIONS = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}

# Original dataset labels → unified labels
OLD2NEW = {
    "E1": "anger_fear",       # Angry
    "E2": "sadness",          # Sadness
    "E3": "anger_fear",       # Fear merged into anger_fear
    "E4": "joy_excited",      # Happiness
    "E5": "calm_content"      # Neutrality
}

file_paths = []
file_emotions = []
data = []

# Loop through actor folders
for actor_folder in os.listdir(KEDAS_DATA_PATH):
    actor_path = os.path.join(KEDAS_DATA_PATH, actor_folder)
    if not os.path.isdir(actor_path):
        continue

    # Loop through audio files inside each actor folder
    for file in os.listdir(actor_path):
        if not (file.endswith(".flac") or file.endswith(".wav")):
            continue

        file_path = os.path.join(actor_path, file)

        # Example: 483-m-01-E1-01.flac
        parts = file.split("-")
        if len(parts) < 5:
            continue  # skip unexpected files

        emotion_old = parts[3]  # E1, E2, E3, E4, E5
        if emotion_old not in OLD2NEW:
            continue

        emotion_new = OLD2NEW[emotion_old]

        # Encode emotion with new mapping
        emotion_encoded = list(KEDAS_EMOTIONS.keys())[list(KEDAS_EMOTIONS.values()).index(emotion_new)]

        file_paths.append(file_path)
        file_emotions.append(emotion_encoded)

        data.append({
            "Name": file_path,
            "Emotion": emotion_encoded
        })

print(f"Loaded {len(data)} files from KEDAS")
KEDAS_Data = pd.DataFrame(data)
KEDAS_Data.to_csv("KEDAS.csv",  index=False)
KEDAS_Data.head()




Loaded 2878 files from KEDAS


,Name,Emotion
0,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,0
1,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,4
2,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,0
3,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,1
4,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,2


### Setting English

In [ ]:
import os
import pandas as pd

RAVD_DATA_PATH = "/content/ravd"

RAVD_EMOTIONS = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}

# Original RAVDESS mapping → unified labels
OLD2NEW = {
    "01": "calm_content",     # neutral
    "02": "calm_content",     # calm
    "03": "joy_excited",      # happy
    "04": "sadness",          # sad
    "05": "anger_fear",       # angry
    "06": "anger_fear",       # fearful
    "07": "curious_reflective", # disgust
    "08": "joy_excited"       # surprised
}

file_names = []
file_emotion = []
file_gender = []
data = []

# Loop through actors
for folder in os.listdir(RAVD_DATA_PATH):
    folder_path = os.path.join(RAVD_DATA_PATH, folder)
    if not os.path.isdir(folder_path):
        continue

    # Loop files
    for file in os.listdir(folder_path):
        if not file.endswith(".wav"):
            continue

        file_path = os.path.join(folder_path, file)
        parts = file.split("-")

        if len(parts) < 7:
            continue

        emotion_old = parts[2]  # third element = emotion code
        actor_id = int(parts[-1].split(".")[0])  # last element = actor ID (before .wav)

        # Map old → new
        if emotion_old not in OLD2NEW:
            continue

        emotion_new = OLD2NEW[emotion_old]
        emotion_encoded = list(RAVD_EMOTIONS.keys())[list(RAVD_EMOTIONS.values()).index(emotion_new)]

        # Gender detection (odd = male, even = female)
        gender = "male" if actor_id % 2 != 0 else "female"

        file_emotion.append(emotion_encoded)
        file_names.append(file)
        file_gender.append(gender)

        data.append({
            "Name": file_path,
            "Emotion": emotion_encoded,
        })

print(f"Loaded {len(data)} files")

# Save to CSV
RAVD_Data = pd.DataFrame(data)
RAVD_Data.to_csv("RAVD.csv", index=False)
RAVD_Data.head()



Loaded 1440 files


,Name,Emotion
0,/content/ravd/Actor_19/03-01-02-01-02-02-19.wav,4
1,/content/ravd/Actor_19/03-01-04-02-01-01-19.wav,2
2,/content/ravd/Actor_19/03-01-08-02-02-02-19.wav,1
3,/content/ravd/Actor_19/03-01-03-01-01-02-19.wav,1
4,/content/ravd/Actor_19/03-01-04-01-02-01-19.wav,2


In [ ]:
import os
import pandas as pd

CREMA_DATA_PATH = "/content/cremad/AudioWAV"

CREMA_EMOTIONS = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}

# Original CREMA labels → unified
OLD2NEW = {
    "ANG": "anger_fear",
    "FEA": "anger_fear",
    "HAP": "joy_excited",
    "SAD": "sadness",
    "DIS": "curious_reflective",
    "NEU": "calm_content"
}

file_names = []
file_emotion = []
file_gender = []  # CREMA has gender in metadata, not filename → leave unknown unless metadata provided
data = []

# Loop through files
for file in os.listdir(CREMA_DATA_PATH):
    if not file.endswith(".wav"):
        continue

    file_path = os.path.join(CREMA_DATA_PATH, file)
    parts = file.split("_")

    if len(parts) < 4:
        continue

    # Extract emotion code
    emotion_old = parts[2]
    if emotion_old not in OLD2NEW:
        continue

    # Map to unified system
    emotion_new = OLD2NEW[emotion_old]
    emotion_encoded = list(CREMA_EMOTIONS.keys())[list(CREMA_EMOTIONS.values()).index(emotion_new)]

    file_emotion.append(emotion_encoded)
    file_names.append(file)
    file_gender.append("unknown")  # need metadata to assign gender

    data.append({
        "Name": file_path,
        "Emotion": emotion_encoded,
    })

print(f"Loaded {len(data)} files")

# Save to CSV
CREMA_Data = pd.DataFrame(data)
CREMA_Data.to_csv("CREMA.csv", index=False)
CREMA_Data.head()


Loaded 7442 files


,Name,Emotion
0,/content/cremad/AudioWAV/1085_MTI_HAP_XX.wav,1
1,/content/cremad/AudioWAV/1013_DFA_HAP_XX.wav,1
2,/content/cremad/AudioWAV/1007_IWW_FEA_XX.wav,0
3,/content/cremad/AudioWAV/1091_IWW_HAP_XX.wav,1
4,/content/cremad/AudioWAV/1006_IEO_ANG_LO.wav,0


In [ ]:
import os
import pandas as pd

# Path to your TESS dataset
TESS_DATA_PATH = "/content/tess/TESS Toronto emotional speech set data"

# Target emotion mapping
TARGET_EMOTIONS = {
    "anger": 0,   # anger_fear
    "fear": 0,    # anger_fear
    "happy": 1,   # joy_excited
    "pleasant_surprise": 1,   # joy_excited
    "sad": 2,     # sadness
    "disgust": 3, # curious_reflective
    "neutral": 4  # calm_content
}

# Containers
file_names = []
file_emotion = []
file_gender = []
data = []

# Loop through dataset
for folder in os.listdir(TESS_DATA_PATH):
    folder_path = os.path.join(TESS_DATA_PATH, folder)
    if not os.path.isdir(folder_path):
        continue

    for file in os.listdir(folder_path):
        if not file.endswith(".wav"):
            continue

        file_path = os.path.join(folder_path, file)

        # Detect emotion from folder name
        emotion = None
        for e in TARGET_EMOTIONS.keys():
            if e.lower() in folder.lower():
                emotion = TARGET_EMOTIONS[e]
                break

        if emotion is None:
            continue  # Skip unknown emotions

        # Gender (OAF = older adult female, YAF = younger adult female)
        if folder.startswith("OAF"):
            gender = "female"
        elif folder.startswith("YAF"):
            gender = "female"
        else:
            gender = "unknown"

        file_names.append(file)
        file_emotion.append(emotion)
        file_gender.append(gender)

        data.append({
            "Name": file_path,
            "Emotion": emotion,
        })

print(f"Loaded {len(data)} files")

# Convert to DataFrame
TESS_Data = pd.DataFrame(data)
TESS_Data.to_csv("TESS.csv", index=False)
TESS_Data.head()


Loaded 2400 files


,Name,Emotion
0,/content/tess/TESS Toronto emotional speech se...,3
1,/content/tess/TESS Toronto emotional speech se...,3
2,/content/tess/TESS Toronto emotional speech se...,3
3,/content/tess/TESS Toronto emotional speech se...,3
4,/content/tess/TESS Toronto emotional speech se...,3


In [ ]:
df_combined  = pd.concat([KEDAS_Data, eyase_Data, TESS_Data, CREMA_Data, RAVD_Data], ignore_index=True)
df_combined.to_csv('dataset.csv', index=False)

In [ ]:
df_combined.head()

,Name,Emotion
0,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,0
1,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,4
2,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,0
3,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,1
4,/content/drive/MyDrive/usamah/emotion/KEDAS/Da...,2


## Training the code

In [ ]:
import os
import torch
import pandas as pd
import torchaudio
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments,
    Trainer,
    Wav2Vec2Config  # Add import for configuration
)
import evaluate
from dataclasses import dataclass
from typing import Optional, Union
import warnings
import random
from torchaudio import transforms as T
from torchaudio.sox_effects import apply_effects_tensor
import librosa
warnings.filterwarnings('ignore')

# Set device for GPU optimization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Enable cuDNN benchmark for faster training
torch.backends.cudnn.benchmark = True

# =====================
# Load Combined Dataset
# =====================
df = pd.read_csv("dataset.csv")  # columns: Name, Emotion

# Check class distribution
print("Class distribution:")
print(df["Emotion"].value_counts())

unique_labels = sorted(df["Emotion"].unique())
label2id = {str(label): i for i, label in enumerate(unique_labels)}
id2label = {i: str(label) for i, label in enumerate(unique_labels)}

print(f"Labels: {label2id}")

df["Emotion"] = df["Emotion"].map(lambda x: label2id[str(x)])

# Stratified split by emotion
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["Emotion"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df["Emotion"], random_state=42)

print(f"Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True))
})

# =====================
# Load Feature Extractor for Large Model
# =====================
model_name = "facebook/wav2vec2-large"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)

# =====================
# Audio Augmentation Functions
# =====================
class AudioAugmentation:
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
        self.noise_factor = 0.005
        self.time_stretch_rates = [0.8, 0.9, 1.1, 1.2]
        self.pitch_shift_steps = [-2, -1, 1, 2]

    def add_noise(self, audio):
        noise = torch.randn_like(audio) * self.noise_factor
        return audio + noise

    def time_stretch(self, audio):
        rate = random.choice(self.time_stretch_rates)
        # Time stretching using Librosa
        audio_np = audio.numpy()
        stretched = librosa.effects.time_stretch(audio_np, rate=rate)
        return torch.from_numpy(stretched)

    def pitch_shift(self, audio):
        steps = random.choice(self.pitch_shift_steps)
        # Pitch shifting using Librosa
        audio_np = audio.numpy()
        shifted = librosa.effects.pitch_shift(audio_np, sr=self.sample_rate, n_steps=steps)
        return torch.from_numpy(shifted)

    def apply_random_eq(self, audio):
        # Simple equalization effect
        freq = random.choice([100, 200, 500, 1000, 2000, 4000])
        gain = random.choice([-12, -6, 0, 6, 12])

        effects = [
            ["equalizer", str(freq), "0.7q", str(gain)]
        ]

        augmented, _ = apply_effects_tensor(audio.unsqueeze(0), self.sample_rate, effects)
        return augmented.squeeze(0)

    def augment(self, audio):
        # Apply a random augmentation
        augmentations = [
            self.add_noise,
            self.time_stretch,
            self.pitch_shift,
            self.apply_random_eq,
        ]

        # Apply 1-2 random augmentations
        num_augmentations = random.randint(1, 2)
        for _ in range(num_augmentations):
            aug_func = random.choice(augmentations)
            try:
                audio = aug_func(audio)
            except:
                # If augmentation fails, continue with original audio
                pass

        return audio

# Initialize audio augmentation
audio_augmenter = AudioAugmentation()

# =====================
# Preprocessing function with augmentation
# =====================
def preprocess(batch, augment=False):
    try:
        speech_array, sr = torchaudio.load(batch["Name"])
        if speech_array.shape[0] > 1:  # stereo → mono
            speech_array = torch.mean(speech_array, dim=0, keepdim=True)
        resampler = torchaudio.transforms.Resample(sr, 16000)
        speech = resampler(speech_array).squeeze()

        # Apply augmentation only for training
        if augment and random.random() > 0.5:  # Apply augmentation 50% of the time
            speech = audio_augmenter.augment(speech)

        speech = speech.numpy()

        # Process without padding first
        inputs = feature_extractor(
            speech,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True  # Add padding here
        )
        batch["input_values"] = inputs.input_values[0]
        batch["labels"] = batch["Emotion"]
        return batch
    except Exception as e:
        print(f"Error processing {batch['Name']}: {e}")
        return None

# Preprocess datasets
def preprocess_train(batch):
    return preprocess(batch, augment=True)

def preprocess_eval(batch):
    return preprocess(batch, augment=False)

# Apply preprocessing
print("Preprocessing training data...")
dataset["train"] = dataset["train"].map(preprocess_train, remove_columns=["Name", "Emotion"])
print("Preprocessing validation data...")
dataset["validation"] = dataset["validation"].map(preprocess_eval, remove_columns=["Name", "Emotion"])
print("Preprocessing test data...")
dataset["test"] = dataset["test"].map(preprocess_eval, remove_columns=["Name", "Emotion"])

# Filter out None values
dataset = dataset.filter(lambda example: example is not None)

# =====================
# Custom Data Collator
# =====================
@dataclass
class DataCollatorCTCWithPadding:
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = True

    def __call__(self, features):
        # Separate input_values and labels
        input_values = [{"input_values": feature["input_values"]} for feature in features]
        labels = [feature["labels"] for feature in features]

        # Pad inputs
        batch = self.feature_extractor.pad(
            input_values,
            padding=self.padding,
            return_tensors="pt",
        )

        # Add labels
        batch["labels"] = torch.tensor(labels, dtype=torch.long)

        # Move to GPU
        if torch.cuda.is_available():
            batch = {k: v.to(device) for k, v in batch.items()}

        return batch

data_collator = DataCollatorCTCWithPadding(feature_extractor=feature_extractor, padding=True)

# =====================
# Model - Large Version with GPU Optimization
# =====================
num_labels = len(label2id)

# First load the config and modify it
config = Wav2Vec2Config.from_pretrained(
    model_name,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    attention_dropout=0.2,
    hidden_dropout=0.2,
    # classifier_dropout is not a valid parameter for Wav2Vec2Config, so remove it
)

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name,
    config=config  # Use the modified config
)

# Move model to GPU
model.to(device)

# Memory optimization for large model
model.config.apply_spec_augment = False

# Freeze most layers initially
for param in model.wav2vec2.parameters():
    param.requires_grad = False

# Unfreeze the last few layers
for layer in model.wav2vec2.encoder.layers[-6:]:
    for param in layer.parameters():
        param.requires_grad = True

# Also unfreeze the classifier
for param in model.classifier.parameters():
    param.requires_grad = True

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# =====================
# TrainingArguments with GPU Optimization
# =====================
training_args = TrainingArguments(
    output_dir="./results_multilingual",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    warmup_ratio=0.1,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
    bf16=torch.cuda.is_bf16_supported(),
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    report_to="tensorboard",
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    max_grad_norm=1.0,
    dataloader_num_workers=4,
    dataloader_prefetch_factor=2,
    remove_unused_columns=False,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    logging_first_step=True,
    group_by_length=True,
)

# =====================
# Metrics
# =====================
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    f1_score = f1.compute(predictions=preds, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

# =====================
# Trainer with GPU Optimization
# =====================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# =====================
# Train with Progressive Unfreezing
# =====================
print("Starting training with multilingual data...")
trainer.train()

# After initial training, unfreeze more layers gradually
print("Unfreezing more layers...")
for param in model.wav2vec2.encoder.layers[-12:].parameters():
    param.requires_grad = True

# Continue training with a lower learning rate
trainer.args.learning_rate = 1e-5
trainer.train()

# Final unfreezing
print("Unfreezing all layers...")
for param in model.wav2vec2.parameters():
    param.requires_grad = True

# Final training with very low learning rate
trainer.args.learning_rate = 5e-6
trainer.train()

# =====================
# Evaluate
# =====================
print("Final evaluation:")
results = trainer.evaluate(dataset["test"])
print(results)

# Save the model
trainer.save_model("./multilingual_emotion_model")
feature_extractor.save_pretrained("./multilingual_emotion_model")

print("Training completed! Model saved to './multilingual_emotion_model'")

# Print GPU memory usage
if torch.cuda.is_available():
    print(f"GPU Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU Memory reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

In [ ]:
import os
import torch
import pandas as pd
import torchaudio
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments,
    Trainer,
    Wav2Vec2Config
)
import evaluate
from dataclasses import dataclass
from typing import Optional, Union
import warnings
import random
from torchaudio import transforms as T
from torchaudio.sox_effects import apply_effects_tensor
import librosa
warnings.filterwarnings('ignore')

# Set device for GPU optimization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Enable cuDNN benchmark for faster training
torch.backends.cudnn.benchmark = True

# =====================
# Load Combined Dataset
# =====================
df = pd.read_csv("dataset.csv")  # columns: Name, Emotion

# Check class distribution
print("Class distribution:")
print(df["Emotion"].value_counts())

unique_labels = sorted(df["Emotion"].unique())
label2id = {str(label): i for i, label in enumerate(unique_labels)}
id2label = {i: str(label) for i, label in enumerate(unique_labels)}

print(f"Labels: {label2id}")

df["Emotion"] = df["Emotion"].map(lambda x: label2id[str(x)])

# Stratified split by emotion
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["Emotion"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df["Emotion"], random_state=42)

print(f"Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True))
})

# =====================
# Load Feature Extractor for Large Model
# =====================
model_name = "facebook/wav2vec2-large"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)

# =====================
# Audio Augmentation Functions
# =====================
class AudioAugmentation:
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
        self.noise_factor = 0.005
        self.time_stretch_rates = [0.8, 0.9, 1.1, 1.2]
        self.pitch_shift_steps = [-2, -1, 1, 2]

    def add_noise(self, audio):
        noise = torch.randn_like(audio) * self.noise_factor
        return audio + noise

    def time_stretch(self, audio):
        rate = random.choice(self.time_stretch_rates)
        # Time stretching using Librosa
        audio_np = audio.numpy()
        stretched = librosa.effects.time_stretch(audio_np, rate=rate)
        return torch.from_numpy(stretched)

    def pitch_shift(self, audio):
        steps = random.choice(self.pitch_shift_steps)
        # Pitch shifting using Librosa
        audio_np = audio.numpy()
        shifted = librosa.effects.pitch_shift(audio_np, sr=self.sample_rate, n_steps=steps)
        return torch.from_numpy(shifted)

    def apply_random_eq(self, audio):
        # Simple equalization effect
        freq = random.choice([100, 200, 500, 1000, 2000, 4000])
        gain = random.choice([-12, -6, 0, 6, 12])

        effects = [
            ["equalizer", str(freq), "0.7q", str(gain)]
        ]

        augmented, _ = apply_effects_tensor(audio.unsqueeze(0), self.sample_rate, effects)
        return augmented.squeeze(0)

    def augment(self, audio):
        # Apply a random augmentation
        augmentations = [
            self.add_noise,
            self.time_stretch,
            self.pitch_shift,
            self.apply_random_eq,
        ]

        # Apply 1-2 random augmentations
        num_augmentations = random.randint(1, 2)
        for _ in range(num_augmentations):
            aug_func = random.choice(augmentations)
            try:
                audio = aug_func(audio)
            except:
                # If augmentation fails, continue with original audio
                pass

        return audio

# Initialize audio augmentation
audio_augmenter = AudioAugmentation()

# =====================
# Preprocessing function with augmentation
# =====================
def preprocess(batch, augment=False):
    try:
        speech_array, sr = torchaudio.load(batch["Name"])
        if speech_array.shape[0] > 1:  # stereo → mono
            speech_array = torch.mean(speech_array, dim=0, keepdim=True)
        resampler = torchaudio.transforms.Resample(sr, 16000)
        speech = resampler(speech_array).squeeze()

        # Apply augmentation only for training
        if augment and random.random() > 0.5:  # Apply augmentation 50% of the time
            speech = audio_augmenter.augment(speech)

        speech = speech.numpy()

        # Process without padding first
        inputs = feature_extractor(
            speech,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True  # Add padding here
        )
        batch["input_values"] = inputs.input_values[0]
        batch["labels"] = batch["Emotion"]
        return batch
    except Exception as e:
        print(f"Error processing {batch['Name']}: {e}")
        return None

# Preprocess datasets
def preprocess_train(batch):
    return preprocess(batch, augment=True)

def preprocess_eval(batch):
    return preprocess(batch, augment=False)

# Apply preprocessing
print("Preprocessing training data...")
dataset["train"] = dataset["train"].map(preprocess_train, remove_columns=["Name", "Emotion"])
print("Preprocessing validation data...")
dataset["validation"] = dataset["validation"].map(preprocess_eval, remove_columns=["Name", "Emotion"])
print("Preprocessing test data...")
dataset["test"] = dataset["test"].map(preprocess_eval, remove_columns=["Name", "Emotion"])

# Filter out None values
dataset = dataset.filter(lambda example: example is not None)

# =====================
# Custom Data Collator (Fixed for CUDA multiprocessing)
# =====================
@dataclass
class DataCollatorCTCWithPadding:
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = True

    def __call__(self, features):
        # Separate input_values and labels
        input_values = [{"input_values": feature["input_values"]} for feature in features]
        labels = [feature["labels"] for feature in features]

        # Pad inputs
        batch = self.feature_extractor.pad(
            input_values,
            padding=self.padding,
            return_tensors="pt",
        )

        # Add labels
        batch["labels"] = torch.tensor(labels, dtype=torch.long)

        # Don't move to GPU here - let the trainer handle device placement
        return batch

data_collator = DataCollatorCTCWithPadding(feature_extractor=feature_extractor, padding=True)

# =====================
# Model - Large Version with GPU Optimization
# =====================
num_labels = len(label2id)

# First load the config and modify it
config = Wav2Vec2Config.from_pretrained(
    model_name,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    attention_dropout=0.2,
    hidden_dropout=0.2,
)

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name,
    config=config  # Use the modified config
)

# Move model to GPU
model.to(device)

# Memory optimization for large model
model.config.apply_spec_augment = False

# Freeze most layers initially
for param in model.wav2vec2.parameters():
    param.requires_grad = False

# Unfreeze the last few layers
for layer in model.wav2vec2.encoder.layers[-6:]:
    for param in layer.parameters():
        param.requires_grad = True

# Also unfreeze the classifier
for param in model.classifier.parameters():
    param.requires_grad = True

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# =====================
# TrainingArguments with GPU Optimization (Fixed for CUDA multiprocessing)
# =====================
training_args = TrainingArguments(
    output_dir="./results_multilingual",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    warmup_ratio=0.1,
    logging_dir="./logs",
    logging_steps=10,
    fp16=not torch.cuda.is_bf16_supported(),  # Use fp16 only if bf16 is not supported
    bf16=torch.cuda.is_bf16_supported(),  # Use bf16 if supported
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    report_to="tensorboard",
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    max_grad_norm=1.0,
    dataloader_num_workers=0,  # Set to 0 to avoid CUDA multiprocessing issues
    dataloader_prefetch_factor=None,  # Remove prefetch factor when workers=0
    remove_unused_columns=False,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    logging_first_step=True,
    group_by_length=False,  # Disable length-based grouping for audio data
)

# =====================
# Metrics
# =====================
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    f1_score = f1.compute(predictions=preds, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

# =====================
# Trainer with GPU Optimization
# =====================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,  # Add tokenizer to handle input_values
)

# =====================
# Train with Progressive Unfreezing
# =====================
print("Starting training with multilingual data...")
trainer.train()

# After initial training, unfreeze more layers gradually
print("Unfreezing more layers...")
for param in model.wav2vec2.encoder.layers[-12:].parameters():
    param.requires_grad = True

# Continue training with a lower learning rate
trainer.create_optimizer_and_scheduler(num_training_steps=len(dataset["train"]) * 5)  # Recreate optimizer with new LR
trainer.args.learning_rate = 1e-5
trainer.train()

# Final unfreezing
print("Unfreezing all layers...")
for param in model.wav2vec2.parameters():
    param.requires_grad = True

# Final training with very low learning rate
trainer.create_optimizer_and_scheduler(num_training_steps=len(dataset["train"]) * 5)  # Recreate optimizer with new LR
trainer.args.learning_rate = 5e-6
trainer.train()

# =====================
# Evaluate
# =====================
print("Final evaluation:")
results = trainer.evaluate(dataset["test"])
print(results)

# Save the model
trainer.save_model("./multilingual_emotion_model")
feature_extractor.save_pretrained("./multilingual_emotion_model")

print("Training completed! Model saved to './multilingual_emotion_model'")

# Print GPU memory usage
if torch.cuda.is_available():
    print(f"GPU Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU Memory reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

Using device: cuda
Class distribution:
Emotion
0    4627
1    3162
2    2586
4    2501
3    1863
Name: count, dtype: int64
Labels: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}
Train=9432, Val=2359, Test=2948
Preprocessing training data...


Map:   0%|          | 0/9432 [00:00<?, ? examples/s]

Preprocessing validation data...


Map:   0%|          | 0/2359 [00:00<?, ? examples/s]

Preprocessing test data...


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.639200,0.797259,0.729123,0.719221
2,0.676900,0.859727,0.714710,0.702501
3,0.795800,0.706617,0.729546,0.721739
4,0.551600,0.836370,0.721916,0.711154
5,0.634400,0.821189,0.710047,0.697621
6,0.694800,0.783781,0.738025,0.728075
7,0.486600,0.823839,0.739296,0.728853
8,0.654000,0.804231,0.727851,0.714338
9,0.509600,0.930554,0.726579,0.715663
10,0.611600,0.906671,0.738872,0.728835


Unfreezing all layers...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.386500,0.922023,0.740992,0.731027
2,0.521100,1.013892,0.737177,0.726708
3,0.456700,0.870113,0.749470,0.741161
4,0.367500,0.952217,0.745655,0.736555
5,0.533500,0.916506,0.742688,0.731078
6,0.531300,1.095439,0.729123,0.714263
7,0.432700,0.999249,0.731666,0.721662
8,0.498600,0.894689,0.751166,0.744136
9,0.505700,1.156217,0.725307,0.711122
10,0.586100,0.955960,0.739296,0.730919


Final evaluation:


{'eval_loss': 0.9368549585342407, 'eval_accuracy': 0.755427408412483, 'eval_f1': 0.7467175588921587, 'eval_runtime': 68.111, 'eval_samples_per_second': 43.282, 'eval_steps_per_second': 5.418, 'epoch': 25.0}
Training completed! Model saved to './multilingual_emotion_model'
GPU Memory allocated: 2.93 GB
GPU Memory reserved: 3.59 GB


## Testing the code

In [ ]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import torchaudio
import numpy as np

from IPython.display import Audio

audio_file_path = "/content/Fear.ogg"


# Load the trained model and feature extractor
model_path = "./multilingual_emotion_model"  # Path where you saved the model
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the label mapping
id2label = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}


# Preprocess the audio file
try:
    speech_array, sr = torchaudio.load(audio_file_path)
    if speech_array.shape[0] > 1:  # stereo → mono
        speech_array = torch.mean(speech_array, dim=0, keepdim=True)
    resampler = torchaudio.transforms.Resample(sr, 16000)
    speech = resampler(speech_array).squeeze().numpy()

    inputs = feature_extractor(
        speech,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Perform prediction
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get the predicted label
    predicted_id = torch.argmax(logits, dim=-1).item()
    predicted_emotion = id2label[predicted_id]

    print(f"The predicted emotion for {audio_file_path} is: {predicted_emotion}")

except Exception as e:
    print(f"Error processing audio file {audio_file_path}: {e}")

try:
    display(Audio(audio_file_path))
except Exception as e:
    print(f"Error loading or playing audio file {audio_file_path}: {e}")


The predicted emotion for /content/Fear.ogg is: anger_fear


In [ ]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import torchaudio
import numpy as np

from IPython.display import Audio

audio_file_path = "/content/Sad.ogg"


# Load the trained model and feature extractor
model_path = "./multilingual_emotion_model"  # Path where you saved the model
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the label mapping
id2label = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}


# Preprocess the audio file
try:
    speech_array, sr = torchaudio.load(audio_file_path)
    if speech_array.shape[0] > 1:  # stereo → mono
        speech_array = torch.mean(speech_array, dim=0, keepdim=True)
    resampler = torchaudio.transforms.Resample(sr, 16000)
    speech = resampler(speech_array).squeeze().numpy()

    inputs = feature_extractor(
        speech,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Perform prediction
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get the predicted label
    predicted_id = torch.argmax(logits, dim=-1).item()
    predicted_emotion = id2label[predicted_id]

    print(f"The predicted emotion for {audio_file_path} is: {predicted_emotion}")

except Exception as e:
    print(f"Error processing audio file {audio_file_path}: {e}")

try:
    display(Audio(audio_file_path))
except Exception as e:
    print(f"Error loading or playing audio file {audio_file_path}: {e}")


The predicted emotion for /content/Sad.ogg is: sadness


In [ ]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import torchaudio
import numpy as np

from IPython.display import Audio

audio_file_path = "/content/Anger.ogg"


# Load the trained model and feature extractor
model_path = "./multilingual_emotion_model"  # Path where you saved the model
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the label mapping
id2label = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}


# Preprocess the audio file
try:
    speech_array, sr = torchaudio.load(audio_file_path)
    if speech_array.shape[0] > 1:  # stereo → mono
        speech_array = torch.mean(speech_array, dim=0, keepdim=True)
    resampler = torchaudio.transforms.Resample(sr, 16000)
    speech = resampler(speech_array).squeeze().numpy()

    inputs = feature_extractor(
        speech,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Perform prediction
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get the predicted label
    predicted_id = torch.argmax(logits, dim=-1).item()
    predicted_emotion = id2label[predicted_id]

    print(f"The predicted emotion for {audio_file_path} is: {predicted_emotion}")

except Exception as e:
    print(f"Error processing audio file {audio_file_path}: {e}")

try:
    display(Audio(audio_file_path))
except Exception as e:
    print(f"Error loading or playing audio file {audio_file_path}: {e}")


The predicted emotion for /content/Anger.ogg is: anger_fear


In [ ]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import torchaudio
import numpy as np

from IPython.display import Audio

audio_file_path = "/content/Natural.wav"


# Load the trained model and feature extractor
model_path = "./multilingual_emotion_model"  # Path where you saved the model
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the label mapping
id2label = {
    0: "anger_fear",
    1: "joy_excited",
    2: "sadness",
    3: "curious_reflective",
    4: "calm_content"
}


# Preprocess the audio file
try:
    speech_array, sr = torchaudio.load(audio_file_path)
    if speech_array.shape[0] > 1:  # stereo → mono
        speech_array = torch.mean(speech_array, dim=0, keepdim=True)
    resampler = torchaudio.transforms.Resample(sr, 16000)
    speech = resampler(speech_array).squeeze().numpy()

    inputs = feature_extractor(
        speech,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Perform prediction
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get the predicted label
    predicted_id = torch.argmax(logits, dim=-1).item()
    predicted_emotion = id2label[predicted_id]

    print(f"The predicted emotion for {audio_file_path} is: {predicted_emotion}")

except Exception as e:
    print(f"Error processing audio file {audio_file_path}: {e}")

try:
    display(Audio(audio_file_path))
except Exception as e:
    print(f"Error loading or playing audio file {audio_file_path}: {e}")


The predicted emotion for /content/Natural.wav is: calm_content


In [60]:
import opensmile
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

class FrequencyAnalyzer:
    """Real-time frequency analyzer for audio"""

    def __init__(self):
        # Initialize OpenSMILE with emobase feature set
        self.smile = opensmile.Smile(
            feature_set=opensmile.FeatureSet.emobase,
            feature_level=opensmile.FeatureLevel.Functionals
        )

    def _find_f0_hz(self, features: pd.DataFrame) -> float:
        """Extract an F0 value (Hz) from OpenSMILE features"""
        if features is None or features.empty:
            return float('nan')

        # Direct Hz candidates
        direct_candidates = [
            'F0_sma_amean',
            'F0final_sma_amean',
            'F0final_sma3_amean',
            'F0_sma_de_amean',
        ]
        for col in direct_candidates:
            if col in features.columns:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    pass

        # Semitone-based candidates
        semi_candidates = [
            'F0semitoneFrom27.5Hz_sma3nz_amean',
            'F0semitoneFrom27.5Hz_sma_amean',
        ]
        for col in semi_candidates:
            if col in features.columns:
                try:
                    semi = float(features[col].iloc[0])
                    if np.isfinite(semi):
                        return 27.5 * (2 ** (semi / 12.0))
                except Exception:
                    pass

        # Generic search
        for col in features.columns:
            name = str(col)
            if 'F0' in name and 'amean' in name:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    continue

        return float('nan')


class Wav2Vec2EmotionAnalyzer:
    """Emotion analyzer using your trained Wav2Vec2 model"""

    def __init__(self, model_path: str = "/content/multilingual_emotion_model"):
        # Load the trained model and feature extractor
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

        # Set the device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Define the label mapping
        self.id2label = {
            0: "anger_fear",
            1: "joy_excited",
            2: "sadness",
            3: "curious_reflective",
            4: "calm_content"
        }

    def analyze_emotion(self, audio_path: str) -> str:
        """Analyze emotion from an audio file using Wav2Vec2"""
        try:
            # Load and preprocess the audio file
            speech_array, sr = torchaudio.load(audio_path)

            # Convert stereo to mono if needed
            if speech_array.shape[0] > 1:
                speech_array = torch.mean(speech_array, dim=0, keepdim=True)

            # Resample to 16kHz if needed
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                speech_array = resampler(speech_array)

            # Extract features
            speech = speech_array.squeeze().numpy()
            inputs = self.feature_extractor(
                speech,
                sampling_rate=16000,
                return_tensors="pt",
                padding=True
            )

            # Move inputs to the same device as the model
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Perform prediction
            with torch.no_grad():
                logits = self.model(**inputs).logits

            # Get the predicted label
            predicted_id = torch.argmax(logits, dim=-1).item()
            return self.id2label[predicted_id]

        except Exception as e:
            print(f"Error in Wav2Vec2 emotion analysis: {e}")
            return "Unknown"


class CombinedAudioAnalyzer:
    """Combines frequency analysis and Wav2Vec2 emotion recognition"""

    def __init__(self, wav2vec_model_path: str = "/content/multilingual_emotion_model"):
        self.frequency_analyzer = FrequencyAnalyzer()
        self.emotion_analyzer = Wav2Vec2EmotionAnalyzer(wav2vec_model_path)

    def analyze_audio_file(self, audio_file_path: str):
        """Analyze an audio file with both frequency and emotion analysis"""
        try:
            # Load the audio file
            data, sample_rate = sf.read(audio_file_path)

            # Ensure proper shape for OpenSMILE
            if data.ndim == 1:
                signal = data.reshape(1, -1)
            elif data.ndim == 2 and data.shape[1] > data.shape[0]:
                signal = data.T
            else:
                signal = data

            # Process with OpenSMILE for frequency analysis
            features = self.frequency_analyzer.smile.process_signal(
                signal=signal,
                sampling_rate=sample_rate
            )

            # Get frequency
            frequency = self.frequency_analyzer._find_f0_hz(features)

            # Analyze emotions with Wav2Vec2
            emotion = self.emotion_analyzer.analyze_emotion(audio_file_path)

            # Print only the required information
            print(f"Predicted Emotion: {emotion}")
            print(f"Frequency: {frequency:.2f} Hz")

        except Exception as e:
            print(f"Error analyzing file: {e}")


def main():
    """Main function to run the audio analysis"""
    # Initialize the analyzer with your model
    analyzer = CombinedAudioAnalyzer("/content/multilingual_emotion_model")

    # Analyze the audio file
    analyzer.analyze_audio_file("/content/Fear.ogg")


if __name__ == "__main__":
    main()

Predicted Emotion: anger_fear
Frequency: 144.06 Hz


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [61]:
import opensmile
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

class FrequencyAnalyzer:
    """Real-time frequency analyzer for audio"""

    def __init__(self):
        # Initialize OpenSMILE with emobase feature set
        self.smile = opensmile.Smile(
            feature_set=opensmile.FeatureSet.emobase,
            feature_level=opensmile.FeatureLevel.Functionals
        )

    def _find_f0_hz(self, features: pd.DataFrame) -> float:
        """Extract an F0 value (Hz) from OpenSMILE features"""
        if features is None or features.empty:
            return float('nan')

        # Direct Hz candidates
        direct_candidates = [
            'F0_sma_amean',
            'F0final_sma_amean',
            'F0final_sma3_amean',
            'F0_sma_de_amean',
        ]
        for col in direct_candidates:
            if col in features.columns:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    pass

        # Semitone-based candidates
        semi_candidates = [
            'F0semitoneFrom27.5Hz_sma3nz_amean',
            'F0semitoneFrom27.5Hz_sma_amean',
        ]
        for col in semi_candidates:
            if col in features.columns:
                try:
                    semi = float(features[col].iloc[0])
                    if np.isfinite(semi):
                        return 27.5 * (2 ** (semi / 12.0))
                except Exception:
                    pass

        # Generic search
        for col in features.columns:
            name = str(col)
            if 'F0' in name and 'amean' in name:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    continue

        return float('nan')


class Wav2Vec2EmotionAnalyzer:
    """Emotion analyzer using your trained Wav2Vec2 model"""

    def __init__(self, model_path: str = "/content/multilingual_emotion_model"):
        # Load the trained model and feature extractor
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

        # Set the device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Define the label mapping
        self.id2label = {
            0: "anger_fear",
            1: "joy_excited",
            2: "sadness",
            3: "curious_reflective",
            4: "calm_content"
        }

    def analyze_emotion(self, audio_path: str) -> str:
        """Analyze emotion from an audio file using Wav2Vec2"""
        try:
            # Load and preprocess the audio file
            speech_array, sr = torchaudio.load(audio_path)

            # Convert stereo to mono if needed
            if speech_array.shape[0] > 1:
                speech_array = torch.mean(speech_array, dim=0, keepdim=True)

            # Resample to 16kHz if needed
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                speech_array = resampler(speech_array)

            # Extract features
            speech = speech_array.squeeze().numpy()
            inputs = self.feature_extractor(
                speech,
                sampling_rate=16000,
                return_tensors="pt",
                padding=True
            )

            # Move inputs to the same device as the model
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Perform prediction
            with torch.no_grad():
                logits = self.model(**inputs).logits

            # Get the predicted label
            predicted_id = torch.argmax(logits, dim=-1).item()
            return self.id2label[predicted_id]

        except Exception as e:
            print(f"Error in Wav2Vec2 emotion analysis: {e}")
            return "Unknown"


class CombinedAudioAnalyzer:
    """Combines frequency analysis and Wav2Vec2 emotion recognition"""

    def __init__(self, wav2vec_model_path: str = "/content/multilingual_emotion_model"):
        self.frequency_analyzer = FrequencyAnalyzer()
        self.emotion_analyzer = Wav2Vec2EmotionAnalyzer(wav2vec_model_path)

    def analyze_audio_file(self, audio_file_path: str):
        """Analyze an audio file with both frequency and emotion analysis"""
        try:
            # Load the audio file
            data, sample_rate = sf.read(audio_file_path)

            # Ensure proper shape for OpenSMILE
            if data.ndim == 1:
                signal = data.reshape(1, -1)
            elif data.ndim == 2 and data.shape[1] > data.shape[0]:
                signal = data.T
            else:
                signal = data

            # Process with OpenSMILE for frequency analysis
            features = self.frequency_analyzer.smile.process_signal(
                signal=signal,
                sampling_rate=sample_rate
            )

            # Get frequency
            frequency = self.frequency_analyzer._find_f0_hz(features)

            # Analyze emotions with Wav2Vec2
            emotion = self.emotion_analyzer.analyze_emotion(audio_file_path)

            # Print only the required information
            print(f"Predicted Emotion: {emotion}")
            print(f"Frequency: {frequency:.2f} Hz")

        except Exception as e:
            print(f"Error analyzing file: {e}")


def main():
    """Main function to run the audio analysis"""
    # Initialize the analyzer with your model
    analyzer = CombinedAudioAnalyzer("/content/multilingual_emotion_model")

    # Analyze the audio file
    analyzer.analyze_audio_file("/content/Anger.ogg")


if __name__ == "__main__":
    main()

Predicted Emotion: anger_fear
Frequency: 94.93 Hz


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [62]:
import opensmile
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

class FrequencyAnalyzer:
    """Real-time frequency analyzer for audio"""

    def __init__(self):
        # Initialize OpenSMILE with emobase feature set
        self.smile = opensmile.Smile(
            feature_set=opensmile.FeatureSet.emobase,
            feature_level=opensmile.FeatureLevel.Functionals
        )

    def _find_f0_hz(self, features: pd.DataFrame) -> float:
        """Extract an F0 value (Hz) from OpenSMILE features"""
        if features is None or features.empty:
            return float('nan')

        # Direct Hz candidates
        direct_candidates = [
            'F0_sma_amean',
            'F0final_sma_amean',
            'F0final_sma3_amean',
            'F0_sma_de_amean',
        ]
        for col in direct_candidates:
            if col in features.columns:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    pass

        # Semitone-based candidates
        semi_candidates = [
            'F0semitoneFrom27.5Hz_sma3nz_amean',
            'F0semitoneFrom27.5Hz_sma_amean',
        ]
        for col in semi_candidates:
            if col in features.columns:
                try:
                    semi = float(features[col].iloc[0])
                    if np.isfinite(semi):
                        return 27.5 * (2 ** (semi / 12.0))
                except Exception:
                    pass

        # Generic search
        for col in features.columns:
            name = str(col)
            if 'F0' in name and 'amean' in name:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    continue

        return float('nan')


class Wav2Vec2EmotionAnalyzer:
    """Emotion analyzer using your trained Wav2Vec2 model"""

    def __init__(self, model_path: str = "/content/multilingual_emotion_model"):
        # Load the trained model and feature extractor
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

        # Set the device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Define the label mapping
        self.id2label = {
            0: "anger_fear",
            1: "joy_excited",
            2: "sadness",
            3: "curious_reflective",
            4: "calm_content"
        }

    def analyze_emotion(self, audio_path: str) -> str:
        """Analyze emotion from an audio file using Wav2Vec2"""
        try:
            # Load and preprocess the audio file
            speech_array, sr = torchaudio.load(audio_path)

            # Convert stereo to mono if needed
            if speech_array.shape[0] > 1:
                speech_array = torch.mean(speech_array, dim=0, keepdim=True)

            # Resample to 16kHz if needed
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                speech_array = resampler(speech_array)

            # Extract features
            speech = speech_array.squeeze().numpy()
            inputs = self.feature_extractor(
                speech,
                sampling_rate=16000,
                return_tensors="pt",
                padding=True
            )

            # Move inputs to the same device as the model
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Perform prediction
            with torch.no_grad():
                logits = self.model(**inputs).logits

            # Get the predicted label
            predicted_id = torch.argmax(logits, dim=-1).item()
            return self.id2label[predicted_id]

        except Exception as e:
            print(f"Error in Wav2Vec2 emotion analysis: {e}")
            return "Unknown"


class CombinedAudioAnalyzer:
    """Combines frequency analysis and Wav2Vec2 emotion recognition"""

    def __init__(self, wav2vec_model_path: str = "/content/multilingual_emotion_model"):
        self.frequency_analyzer = FrequencyAnalyzer()
        self.emotion_analyzer = Wav2Vec2EmotionAnalyzer(wav2vec_model_path)

    def analyze_audio_file(self, audio_file_path: str):
        """Analyze an audio file with both frequency and emotion analysis"""
        try:
            # Load the audio file
            data, sample_rate = sf.read(audio_file_path)

            # Ensure proper shape for OpenSMILE
            if data.ndim == 1:
                signal = data.reshape(1, -1)
            elif data.ndim == 2 and data.shape[1] > data.shape[0]:
                signal = data.T
            else:
                signal = data

            # Process with OpenSMILE for frequency analysis
            features = self.frequency_analyzer.smile.process_signal(
                signal=signal,
                sampling_rate=sample_rate
            )

            # Get frequency
            frequency = self.frequency_analyzer._find_f0_hz(features)

            # Analyze emotions with Wav2Vec2
            emotion = self.emotion_analyzer.analyze_emotion(audio_file_path)

            # Print only the required information
            print(f"Predicted Emotion: {emotion}")
            print(f"Frequency: {frequency:.2f} Hz")

        except Exception as e:
            print(f"Error analyzing file: {e}")


def main():
    """Main function to run the audio analysis"""
    # Initialize the analyzer with your model
    analyzer = CombinedAudioAnalyzer("/content/multilingual_emotion_model")

    # Analyze the audio file
    analyzer.analyze_audio_file("/content/Natural.wav")


if __name__ == "__main__":
    main()

Predicted Emotion: calm_content
Frequency: 45.73 Hz


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [63]:
import opensmile
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

class FrequencyAnalyzer:
    """Real-time frequency analyzer for audio"""

    def __init__(self):
        # Initialize OpenSMILE with emobase feature set
        self.smile = opensmile.Smile(
            feature_set=opensmile.FeatureSet.emobase,
            feature_level=opensmile.FeatureLevel.Functionals
        )

    def _find_f0_hz(self, features: pd.DataFrame) -> float:
        """Extract an F0 value (Hz) from OpenSMILE features"""
        if features is None or features.empty:
            return float('nan')

        # Direct Hz candidates
        direct_candidates = [
            'F0_sma_amean',
            'F0final_sma_amean',
            'F0final_sma3_amean',
            'F0_sma_de_amean',
        ]
        for col in direct_candidates:
            if col in features.columns:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    pass

        # Semitone-based candidates
        semi_candidates = [
            'F0semitoneFrom27.5Hz_sma3nz_amean',
            'F0semitoneFrom27.5Hz_sma_amean',
        ]
        for col in semi_candidates:
            if col in features.columns:
                try:
                    semi = float(features[col].iloc[0])
                    if np.isfinite(semi):
                        return 27.5 * (2 ** (semi / 12.0))
                except Exception:
                    pass

        # Generic search
        for col in features.columns:
            name = str(col)
            if 'F0' in name and 'amean' in name:
                try:
                    val = float(features[col].iloc[0])
                    if np.isfinite(val) and val > 0:
                        return val
                except Exception:
                    continue

        return float('nan')


class Wav2Vec2EmotionAnalyzer:
    """Emotion analyzer using your trained Wav2Vec2 model"""

    def __init__(self, model_path: str = "/content/multilingual_emotion_model"):
        # Load the trained model and feature extractor
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

        # Set the device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Define the label mapping
        self.id2label = {
            0: "anger_fear",
            1: "joy_excited",
            2: "sadness",
            3: "curious_reflective",
            4: "calm_content"
        }

    def analyze_emotion(self, audio_path: str) -> str:
        """Analyze emotion from an audio file using Wav2Vec2"""
        try:
            # Load and preprocess the audio file
            speech_array, sr = torchaudio.load(audio_path)

            # Convert stereo to mono if needed
            if speech_array.shape[0] > 1:
                speech_array = torch.mean(speech_array, dim=0, keepdim=True)

            # Resample to 16kHz if needed
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                speech_array = resampler(speech_array)

            # Extract features
            speech = speech_array.squeeze().numpy()
            inputs = self.feature_extractor(
                speech,
                sampling_rate=16000,
                return_tensors="pt",
                padding=True
            )

            # Move inputs to the same device as the model
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Perform prediction
            with torch.no_grad():
                logits = self.model(**inputs).logits

            # Get the predicted label
            predicted_id = torch.argmax(logits, dim=-1).item()
            return self.id2label[predicted_id]

        except Exception as e:
            print(f"Error in Wav2Vec2 emotion analysis: {e}")
            return "Unknown"


class CombinedAudioAnalyzer:
    """Combines frequency analysis and Wav2Vec2 emotion recognition"""

    def __init__(self, wav2vec_model_path: str = "/content/multilingual_emotion_model"):
        self.frequency_analyzer = FrequencyAnalyzer()
        self.emotion_analyzer = Wav2Vec2EmotionAnalyzer(wav2vec_model_path)

    def analyze_audio_file(self, audio_file_path: str):
        """Analyze an audio file with both frequency and emotion analysis"""
        try:
            # Load the audio file
            data, sample_rate = sf.read(audio_file_path)

            # Ensure proper shape for OpenSMILE
            if data.ndim == 1:
                signal = data.reshape(1, -1)
            elif data.ndim == 2 and data.shape[1] > data.shape[0]:
                signal = data.T
            else:
                signal = data

            # Process with OpenSMILE for frequency analysis
            features = self.frequency_analyzer.smile.process_signal(
                signal=signal,
                sampling_rate=sample_rate
            )

            # Get frequency
            frequency = self.frequency_analyzer._find_f0_hz(features)

            # Analyze emotions with Wav2Vec2
            emotion = self.emotion_analyzer.analyze_emotion(audio_file_path)

            # Print only the required information
            print(f"Predicted Emotion: {emotion}")
            print(f"Frequency: {frequency:.2f} Hz")

        except Exception as e:
            print(f"Error analyzing file: {e}")


def main():
    """Main function to run the audio analysis"""
    # Initialize the analyzer with your model
    analyzer = CombinedAudioAnalyzer("/content/multilingual_emotion_model")

    # Analyze the audio file
    analyzer.analyze_audio_file("/content/Sad.ogg")


if __name__ == "__main__":
    main()

Predicted Emotion: sadness
Frequency: 77.29 Hz


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r